# After Validators

As we saw in the lecture video, a custom validator is essentially a transformation function, that receives an input and needs to return the value that is being deserialized.

We can also use the opportunity to perform validation checks, and raise an exception if appropriate.

So, these custom validators, are not just validators, but are also used to transform the data being deserialized. That's an important point, because they are sometimes used for validation only, sometimes for deserialization only, and sometimes for both.

In this video we're going to start by defining custom validators using a decorator:

In [1]:
from pydantic import BaseModel, Field, field_validator, ValidationError

In this first example, we're just going to make a custom validator that checks that an integer is even.

We could (and probably should) of course just use the `Field` constraint (`multiple_of`), but this will serve to illustrate a simple validator.

The default behavior of validators defined using the decorator approach, is an **AFTER** validator.

What this means, is that the value being piped into the validator, is the one that has already been processed by Pydantic according to the field type and other constraints that were placed on the field.

In [2]:
class Model(BaseModel):
    number: int = Field(gt=0, lt=10)

Notice that we have defined a field that should be an integer, greater than `0`, and less than `10`.

As-is, Pydantic will run it's internal validation and coercion on input data:

In [3]:
Model(number="4")

Model(number=4)

Notice how the string "4" was converted to an integer. Moreover, Pydantic will ensure that the integer number is within the specified bounds.

In [4]:
try:
    Model(number=12)
except ValidationError as ex:
    print(ex)

1 validation error for Model
number
  Input should be less than 10 [type=less_than, input_value=12, input_type=int]
    For further information visit https://errors.pydantic.dev/2.5/v/less_than


Let's now add an after validator for that field.

Note that custom validators are **class** methods, so they will be bound to the class, not the instance, and we have to define the function signature appropriately, as well as mark the function as a class method (don't have to, but better to do so).

In [5]:
class Model(BaseModel):
    number: int = Field(gt=0, lt=10)

    @field_validator("number")
    @classmethod
    def validate_even(cls, value):
        print("Running custom validator")
        print(f"{value=}, {type(value)=}")
        return value  # custom validators must return a value

Now, let's try it with valid input:

In [6]:
Model(number=3)

Running custom validator
value=3, type(value)=<class 'int'>


Model(number=3)

You can see that our validator function was called. Also notice how the `value` argument received by our validator was an integer. Now, you could think that's because we specified an integer when we create a model instance, but look at this:

In [7]:
Model(number="3")

Running custom validator
value=3, type(value)=<class 'int'>


Model(number=3)

You'll notice that our validator still received an integer.

And if we try to specify a value that does not satisfy the Pydantic constraints we placed:

In [8]:
try:
    Model(number=12)
except ValidationError as ex:
    print(ex)

1 validation error for Model
number
  Input should be less than 10 [type=less_than, input_value=12, input_type=int]
    For further information visit https://errors.pydantic.dev/2.5/v/less_than


you'll notice that our custom validator was **NOT** called.

This is how an **after** validator works - it is called, after Pydantic's own validators/deserializers have (succesfully) completed. 

This means that since we defined `number` to be an integer value between 0 and 10, when our validator runs, we are **assured** that `value` will satisfy the type and constraints.

So, our validator just becomes something extra we want to validate, on top of whatever Pydantic validations that have already occurred.

Let's complete our validator.

To indicate a validation error in our custom validator, we just raise a `ValueError` with an appropriate message.

In [9]:
class Model(BaseModel):
    number: int = Field(gt=0, lt=10)

    @field_validator("number")
    @classmethod
    def validate_even(cls, value):
        print("Running custom validator")
        print(f"{value=}, {type(value)=}")
        if value % 2 == 0:
            # number is even, so return it
            return value
        raise ValueError("value must be even")

And let's try it out:

In [10]:
Model(number=4)

Running custom validator
value=4, type(value)=<class 'int'>


Model(number=4)

In [11]:
try:
    Model(number=3)
except ValidationError as ex:
    print(ex)

Running custom validator
value=3, type(value)=<class 'int'>
1 validation error for Model
number
  Value error, value must be even [type=value_error, input_value=3, input_type=int]
    For further information visit https://errors.pydantic.dev/2.5/v/value_error


Notice how the exception that was raised was a `ValidationError` exception, not a `ValueError`. Pydantic will basically catch any `ValueError` in our custom validator, and raise a `ValidationError` in its place.

Other errors, such as `TypeError` for example, are not handled by Pydantic, and will bubble up unaffected:

In [12]:
class Model(BaseModel):
    number: int = Field(gt=0, lt=10)

    @field_validator("number")
    @classmethod
    def validate_even(cls, value):
        print("Running custom validator")
        print(f"{value=}, {type(value)=}")
        if value % 2 == 0:
            # number is even, so return it
            return value
        raise TypeError("value must be even")

In [13]:
try:
    Model(number=3)
except Exception as ex:
    print(f"{type(ex)=}, {ex}")

Running custom validator
value=3, type(value)=<class 'int'>
type(ex)=<class 'TypeError'>, value must be even


As you can see, the `TypeError` we raised in our custom validator bubbled up as a itself, not a validation error. So, to remain consistent with the fact that Pydantic raises `ValidationError`, we raise `ValueError`.

Pydantic documentation clearly states that we should not raise a `ValidationError` ourselves - instead we raise that `ValueError` and Pydantic handles the rest.

As I mentioned earlier, a custom validator is also a good place to modify the value being deserialized if we need to.

Let's look at this simple example, where we want to have an even number for our field, and if the provided number is not even, we want to change it to the next even number.

(A bit of a silly example, and we'll look at something a bit more realistic later - for now this will serve as a simple example).

In [14]:
class Model(BaseModel):
    even: int

    @field_validator("even")
    @classmethod
    def make_even(cls, value: int) -> int:
        if value % 2 == 1:
            return value + 1
        return value

Notice that our validator does not have to check that `value` is an integer - since it is an **after** validator, we are guaranteed that `value` is already an integer.

In [15]:
Model(even="3")

Model(even=4)

Let's look at a slightly more realistic example of using a validator for modifying the value being deserialized.

Suppose, we have a `datetime` object on a model, and we want to always have that value be an aware UTC datetime.

In other words, if the incoming `datetime` value is naive, we assume it is already in UTC, and simply change it to be aware, but if it is aware, we convert it to UTC.

Basically this is similar to the example we looked at when studying custom serializers. But instead of transforming the value during serialization, we are going to do it during deserialization instead.

In [16]:
from datetime import datetime
import pytz

def make_utc(dt: datetime) -> datetime:
    if dt.tzinfo is None:
        dt = pytz.utc.localize(dt)
    else:
        dt = dt.astimezone(pytz.utc)
    return dt

This is the function we used before.

Now, let's make a model that ensures datetimes are always stored as UTC aware datetimes in our model.

In [17]:
class Model(BaseModel):
    dt: datetime

    @field_validator("dt")
    @classmethod
    def make_utc(cls, dt: datetime) -> datetime:
        if dt.tzinfo is None:
            dt = pytz.utc.localize(dt)
        else:
            dt = dt.astimezone(pytz.utc)
        return dt

And let's try it out:

In [18]:
Model(dt="2020-01-01T03:00:00")

Model(dt=datetime.datetime(2020, 1, 1, 3, 0, tzinfo=<UTC>))

And if we use a datetime that is timezone aware:

In [19]:
eastern = pytz.timezone('US/Eastern')
dt = eastern.localize(datetime(2020, 1, 1, 3, 0, 0))

In [20]:
Model(dt=dt)

Model(dt=datetime.datetime(2020, 1, 1, 8, 0, tzinfo=<UTC>))

We can always specify multiple validators for the same field.

As we saw in the lecture video, the order of execution of after validators is the top to bottom definition order of the functions in the class.

In [21]:
class Model(BaseModel):
    number: int

    @field_validator("number")
    @classmethod
    def add_1(cls, value: int):
        print(f"running add_1({value}) -> {value + 1}")
        return value + 1

    @field_validator("number")
    @classmethod
    def add_2(cls, value: int):
        print(f"running add_2({value}) -> {value + 2}")
        return value + 2

    @field_validator("number")
    @classmethod
    def add_3(cls, value: int):
        print(f"running add_3({value}) -> {value + 3}")
        return value + 3


In [22]:
Model(number=1)

running add_1(1) -> 2
running add_2(2) -> 4
running add_3(4) -> 7


Model(number=7)

As you can see, all three custom validators were executed in definition order, top to bottom.

Technically, we can also apply the same validator to multiple fields.

Suppose we have a class with some float values, and we want those floats to always be rounded to decimal points.

We can easily do this with a custom validator:

In [23]:
class Model(BaseModel):
    unit_cost: float
    unit_price: float

    @field_validator("unit_cost", "unit_price")
    @classmethod
    def round_2(cls, value: float) -> float:
        return round(value, 2)

In [24]:
Model(unit_cost=2.12345, unit_price=5.9876)

Model(unit_cost=2.12, unit_price=5.99)

If we wanted our validator to apply to all fields in the class, we could also use the special `"*"` indicator instead of field names:

In [25]:
class Model(BaseModel):
    unit_cost: float
    unit_price: float

    @field_validator("*")
    @classmethod
    def round_2(cls, value: float) -> float:
        return round(value, 2)

In [26]:
Model(unit_cost=2.12345, unit_price=5.9876)

Model(unit_cost=2.12, unit_price=5.99)